# Coversational Interface - Chatbot with Claude LLM
In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Claude 3 as our FM for building the chatbot and InMemoryChatMessageHistory to store the conversation history.

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers. Chatbots uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps.

![alt text](./images/chatbot_bedrock.png)


In [ ]:
%pip install -U langchain-community===0.2.12
%pip install -U \
     "langchain>=0.2.14" \
     "langchain-aws>=0.1.17"

In [ ]:
import boto3
import botocore
import pprint

pp = pprint.PrettyPrinter(indent=2)

### Set up

In [ ]:
boto3_session = boto3.session.Session()
region = boto3_session.region_name

# the statement below can be used to override the region in the session
#region = "us-west-2"

### Run the chatbot

Set up parameters for the model and create a client

In [ ]:
model = "anthropic.claude-3-sonnet-20240229-v1:0"
temperature = 0.1

In [ ]:
from langchain_aws.chat_models import ChatBedrock

llm_chat = ChatBedrock(
    model_id=model, 
    model_kwargs={"temperature": temperature},
    region_name=region
)


Passing conversation state into and out a chain is vital when building a chatbot. The RunnableWithMessageHistory class lets us add message history to certain types of chains. It wraps another Runnable and manages the chat message history for it. Specifically, it loads previous messages in the conversation BEFORE passing it to the Runnable, and it saves the generated response as a message AFTER calling the runnable. This class also enables multiple conversations by saving each conversation with a session_id - it then expects a session_id to be passed in the config when calling the runnable, and uses that to look up the relevant conversation history. You have to implement a function like the get_session_history() which will take as input the session-id and return the conversation for the session. Here we have a simple implementation of the conversation history, using the InMemoryChatMessageHistory class

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm_chat, 
                                                  get_session_history)


In [ ]:
from langchain_core.messages import HumanMessage

with_message_history.invoke(
    [HumanMessage(content="hi - i am bob!")],
    config={"configurable": {"session_id": "1"}},
)

In [ ]:
with_message_history.invoke(
    [HumanMessage(content="whats my name?")],
    config={"configurable": {"session_id": "1"}},
)

At this point the store has 1 key (session_id = '1') and its value is a list with 4 messages:
 [HumanMessage, AIMessage, HumanMessage, AIMessage]

All langchain messages have 3 properties: a role, content, response_metadata. The HumanMessage returns 1 property, the *content* (e.g. the message that the user passed) whereas the AIMessage also returns non-empty *response_metadata*. It also includes the property *usage_metadata*, a dictionary with these keys: input_tokens, output_tokens, total_tokens (these are also included in the response_metadata) 


In [ ]:
print(store)
# uncomment the following line to take a closer look at the message associated with session_id='1'
#store['1'].messages[1]

### Re-implement the chatbot, using a Prompt Template and chaining the prompt with the runnable

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system","You're an assistant who speaks in {language}. Translate the user input"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | llm_chat

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

lang = "French"
pp.pprint(chain_with_history.invoke(
    {"language": lang, "question": "Hi my name is John"},
    config={"configurable": {"session_id": "2"}}
))

In [ ]:
pp.pprint(chain_with_history.invoke(
    {"language": lang, "question": "What is my name?"},
    config={"configurable": {"session_id": "2"}}
))

In [ ]:
pp.pprint(store['2'].messages)

In [ ]:
# take a closer look at the messages kept in the conversation history
pp.pprint(store['2'].messages[0].response_metadata)
pp.pprint(store['2'].messages[1].response_metadata)
pp.pprint(store['2'].messages[1].usage_metadata)